`Generowanie danych dla przyszłych wartości zmiennych objaśniających w modelu. Prognoza ma byś dokłądna kwartalnie, ale prognozować będziemy miesięcznie`

### Biblioteki

In [1]:
# IMPORTOWANIE BIBLIOTEK

from datetime import datetime
import datetime as dt

import math
import numpy as np
import os
import pandas as pd

# do otwarcia zip z url
import requests
from zipfile import ZipFile
from io import BytesIO

`Wczytanie kalendarza`

In [2]:
kalendarz = pd.read_excel('xxx.xlsx', sheet_name= '')
kalendarz = kalendarz[('' > kalendarz.data) & (kalendarz.data > '')]

`Informacje dla sklepów`

In [4]:
sklepy_informacje = pd.read_excel('', sheet_name= '')

In [5]:
# Połączenie kalendarza dla każdego ze sklepów
kalendarz['tmp'] = 1
sklepy_informacje['tmp'] = 1
X = pd.merge(sklepy_informacje, kalendarz, on=['tmp'])
X = X.drop('tmp', axis=1)
X = X.rename(columns = {'data':'CALDAY'})

`Dane makroekonomiczne`

In [ ]:
makroekonomiczne = pd.read_excel('dane_predykcje.xlsx', sheet_name= 'dane_makro').set_index('index').stack().reset_index().rename(columns={'level_1':'kwartał',0:'wartość'})
makroekonomiczne = makroekonomiczne[~makroekonomiczne.kwartał.isin(['2022','2023','2024','2025'])]

# Usunięcie niepotrzebnych spacji w wartościach
makroekonomiczne = makroekonomiczne.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Zamiana tekstu na datę - kwartał
makroekonomiczne['kwartał'] =  pd.to_datetime(makroekonomiczne['kwartał']).dt.to_period('Q')

# Stworzenie ostatecznej tabelki 
makroekonomiczne = makroekonomiczne.pivot_table('wartość', 'kwartał','index').reset_index()

In [11]:
# stworzenie w danych kolumny z kwartałem
X['kwartał'] = pd.PeriodIndex(X.CALDAY, freq='Q')

# połączenie
X = X.merge(makroekonomiczne, on='kwartał', how = 'left')

In [12]:
# Lista do sterowania zmiennymi makroekonomicznycmi
kolumny_makro = ['Wskaźnik_inflacji',
       'Ceny energii (%, r/r)',
       'Ceny żywności (%, r/r)',
       'Deflator wartości dodanej za granicą (%, r/r)', 'Eksport (%, r/r)',
       'Import (%, r/r)',
       'Indeks cen surowców energetycznych (USD; 2011=1,00)',
       'Indeks cen surowców rolnych (EUR; 2011=1,00)',
       'Inflacja bazowa (CPI po wyłączeniu cen żywności i energii, %, r/r)',
       'Jednostkowe koszty pracy (%, r/r)',
       'Luka popytowa (% PKB potencjalnego)', 'NAWRU (%)',
       'Nakłady brutto na środki trwałe (%, r/r)', 'PKB (%, r/r)',
       'PKB za granicą (%, r/r)', 'Popyt krajowy (%, r/r)',
       'Pracujący (%, r/r)', 'Produkt potencjalny (%, r/r)',
       'Saldo rachunku bieżącego (% PKB)',
       'Spożycie gospodarstw domowych (%, r/r)',# 'Spożycie publiczne (%, r/r)',
       'Stopa aktywności zawodowej (%)', 'Stopa bezrobocia (%)',
       'WIBOR 3M (%)', 'Wkład eksportu netto (pkt proc., r/r)',
       'Wskaźnik cen konsumenta CPI (%, r/r)', 'Wydajność pracy (%, r/r)',
       'Wynagrodzenia (%, r/r)']

`Dane pogodowe`

In [13]:
stacje = [100, 105, 115, 120, 125, 135, 155, 160, 185, 195, 200, 205, 210, 230, 235, 250, 270, 272, 280, 295, 300, 310, 330, 360, 375, 385, 399, 400, 415, 418, 424, 435, 455, 465, 469, 488, 495, 497, 500, 510, 520, 530, 540, 550, 560, 566, 570, 575, 580, 585, 595, 600, 625, 628, 650, 660, 670, 690]
miesiące = ['01','02','03','04','05','06','07','08','09','10','11','12']
start = 2018
lata = []
for i in range(5):
    lata.append(start+i)
    

appended_data = []
kolumny_pogoda = pd.read_excel('xxx.xlsx', sheet_name= '', header = None)
kolumny_do_usunięcia = ['Status pomiaru NOS', 'Status pomiaru FWS', 'Status pomiaru TEMP', 'Status pomiaru CPW', 'Status pomiaru WLGS','Status pomiaru PPPS', 'Status pomiaru PPPM', 'Status pomiaru WODZ', 'Status pomiaru WONO']
for i in stacje:
    for ii in lata:
        try:
            r = requests.get(f"https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/{str(ii)}/{str(ii)}_{str(i)}_s.zip")
            files = ZipFile(BytesIO(r.content))
            pogoda = pd.read_csv(files.open(f"s_d_t_{str(i)}_{str(ii)}.csv"), header = None, encoding = 'windows-1250')
            pogoda.columns = list(kolumny_pogoda.iloc[:,0])
            pogoda.drop(kolumny_do_usunięcia, axis=1, inplace=True)
            appended_data.append(pogoda)
        except:
            continue

pogoda = pd.concat(appended_data)

# Errata do danych
'''
Stacja Katowice-Muchowiec do 31.12.2018 to stacja Katowice
Stacja Łóź-Lublinek do 31.12.2018 to stacja Łóź
Stacja Poznań-Ławica do 31.12.2018 to stacja Poznań
Stacja Warszawa-Okęcie do 31.12.2018 to stacja Warszawa
Stacja Wrocław-Strachowice do 31.12.2018 to stacja Wrocław
Stacja Elbląg-Milejewo do 31.03.2013 to stacja Elbląg w inej lokalizacji, system nie pozwala przechowywać podwójnej nazwy stacji dla tego samego kodu.
Stacja Resko-Smólsko do 31.12.2014 to stacja Resko.
Stacja Kołobrzeg-Dźwirzyno do 8.04.2018 to stacja Kołobrzeg, nastąpiła zmiana lokalizacji stacji z zachowaniem kodu.
''' ;    

pogoda['Nazwa stacji'] = pogoda['Nazwa stacji'].apply(lambda x: 'KOŁOBRZEG' if x == 'KOŁOBRZEG-DŹWIRZYNO' else 
                                                      ('WARSZAWA' if x == 'WARSZAWA-OKĘCIE' else
                                                      ('RESKO' if x == 'RESKO-SMÓLSKO' else
                                                      ('ELBLĄG' if x == 'ELBLĄG-MILEJEWO' else
                                                      ('POZNAŃ' if x == 'POZNAŃ-ŁAWICA' else
                                                      ('ŁÓDŹ' if x == 'ŁÓDŹ-LUBLINEK' else
                                                      ('KATOWICE' if x == 'KATOWICE-MUCHOWIEC' else
                                                      ('WROCŁAW' if x == 'WROCŁAW-STRACHOWICE' else x))))))))                                              
                                                

# Obróbka kolumn przed dołączeniem historycznych informacji o pogodzie
pogoda['CALDAY'] = pd.to_datetime(dict(year=pogoda.Rok, month=pogoda.Miesiąc, day=pogoda.Dzień))
pogoda = pogoda.rename(columns = {'Nazwa stacji':'STACJA_POGODOWA'})

# Kolumny do połączenia
pogoda_kolumny = ['Kod stacji', 'STACJA_POGODOWA', 'Średnie dobowe zachmurzenie ogólne [oktanty]', 'Średnia dobowa prędkość wiatru [m/s]',
 'Średnia dobowa temperatura [°C]', 'Średnia dobowa wilgotność względna [%]', 'Średnia dobowe ciśnienie na poziomie stacji [hPa]', 
 'Średnie dobowe ciśnienie na pozimie morza [hPa]', 'Suma opadu dzień  [mm]', 'Suma opadu noc   [mm]', 'CALDAY']


In [14]:
weather_cols_sums = ['Średnie dobowe zachmurzenie ogólne [oktanty]',          
       'Suma opadu dzień  [mm]', 'Suma opadu noc   [mm]']              
weather_cols_means = ['Średnia dobowa prędkość wiatru [m/s]',          
       'Średnia dobowa temperatura [°C]',                              
       'Średnia dobowa wilgotność względna [%]',                       
       'Średnia dobowe ciśnienie na poziomie stacji [hPa]']             
pogoda = pogoda.groupby(['STACJA_POGODOWA','Miesiąc']).aggregate({**{ col : 'mean' for col in weather_cols_means},**{ col : 'mean' for col in weather_cols_sums}}).reset_index()

In [15]:
X['Miesiąc'] = pd.PeriodIndex(X.CALDAY, freq='M').month
X = X.merge(pogoda, on=['STACJA_POGODOWA','Miesiąc'], how = 'left')

`Dane inwentaryzacje`

In [17]:
inwentaryzacje = pd.read_excel('xxx.xlsx', sheet_name= '')
inwentaryzacje['INVENTORY'] = 1

`Zapisanie kroku`

In [25]:
X = X.drop_duplicates()
X.to_csv('Zbiór_X.csv', index = False)